This notebook is part of the supplementary material of the books "Online Machine Learning - Eine praxisorientiere Einführung",  
https://link.springer.com/book/9783658425043 and "Online Machine Learning - A Practical Guide with Examples in Python" https://link.springer.com/book/9789819970063
The contents are open source and published under the "BSD 3-Clause License".
This software is provided "as is" without warranty of any kind, either express or implied, including but not limited to implied warranties of merchantability and fitness for a particular purpose. The author or authors assume no liability for any damages or liability, whether in contract, tort, or otherwise, arising out of or in connection with the software or the use or other dealings with the software.

# Chapter 8: Short Introduction to River

## Batch Learning with `sklearn`

In [1]:
from sklearn import linear_model
from sklearn import metrics
from sklearn import model_selection
from sklearn import pipeline
from sklearn import preprocessing
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### California Housing

* As a simple example of batch learning, suppose we want to learn to predict whether or not the price of a house in California is above the median house price.

### Warning:
There are at least three different variants of the `California Housing` dataset:
1. the `original` dataset (statLib)
2. the `sklearn` dataset
3. the `kaggle` dataset

* We first use the `sklearn` dataset, which can be called as follows:

In [2]:
from sklearn.datasets import fetch_california_housing
import pandas as pd
cal_housing = fetch_california_housing()
features = cal_housing.feature_names
X = pd.DataFrame(cal_housing.data, columns=features)
y = pd.Series(cal_housing.target)
## compute the 95% percentile of the target variable y
y_75 = y.quantile(0.75)
print(f"y_75: {y_75}")
y = y.apply(lambda x: 1 if x > y_75 else 0)

y_75: 2.6472499999999997


* Our goal is to assign a set of characteristics to a binary decision using logistic regression.
* Like many other models based on numerical weights, logistic regression is sensitive to feature scaling. Rescaling the data so that each characteristic has a mean of 0 and a variance of 1 is generally considered best practice. We can apply rescaling and fit the logistic regression sequentially in an elegant way using a pipeline.
* To measure the performance of the model, we evaluate the average accuracy using 5-fold cross-validation.

In [3]:
# Define the steps of the model
model = pipeline.Pipeline([
    ('scale', preprocessing.StandardScaler()),
    ('lin_reg', linear_model.LogisticRegression(solver='lbfgs'))
])

# Define a deterministic cross-validation procedure
cv = model_selection.KFold(n_splits=5, shuffle=True, random_state=42)

# Compute the MSE values
scorer = metrics.make_scorer(metrics.accuracy_score)
scores = model_selection.cross_val_score(model, X, y, scoring=scorer, cv=cv)

# Display the average score and it's standard deviation
print(f'Accuracy: {scores.mean():.3f} (± {scores.std():.3f})')

Accuracy: 0.871 (± 0.005)


## The `river` package

* `river` works "by design" with dictionaries (Python dictionaries), so called `dicts`. 
* The `river` programmers believe that it is more convenient to program with `dicts` than with `numpy.ndarrays`, at least when it comes to single observations:
  * `dicts` have the added advantage that each function can be accessed by name rather than by position.
* Conveniently, river's `stream` module has an `iter_sklearn_dataset` method that we can use to convert `sklearn` data to `river` data.

In [4]:
from river import stream
from sklearn.datasets import fetch_california_housing
import pandas as pd

def get_features():
    california_housing = fetch_california_housing()
    return california_housing.feature_names

def get_dataset():
    california_housing = fetch_california_housing()
    features = california_housing.feature_names
    X = pd.DataFrame(california_housing.data, columns=features)
    y = pd.Series(california_housing.target)
    ## compute the 95% percentile of the target variable y
    y_75 = y.quantile(0.75)
    print(f"y_95: {y_75}")
    y = y.apply(lambda x: 1 if x > y_75 else 0)
    dataset = stream.iter_pandas(X, y)
    return dataset


dataset=get_dataset()

for xi, yi in dataset:    
    print(xi, yi)
    break


y_95: 2.6472499999999997
{'MedInc': 8.3252, 'HouseAge': 41.0, 'AveRooms': 6.984126984126984, 'AveBedrms': 1.0238095238095237, 'Population': 322.0, 'AveOccup': 2.5555555555555554, 'Latitude': 37.88, 'Longitude': -122.23} 1


* Since the algorithms operate on a stream of data, many calculations cannot be performed as in classic batch mode.
  * For example, suppose we want to scale the data so that it has mean 0 and variance 1.  To do this in the BML, we simply need to subtract the mean of each feature from each value and then divide the result by the standard deviation of the feature.  The problem is that we cannot possibly know the values of the mean and standard deviation before we have actually gone through all the data. 
* One approach would be to make a first pass over the data to calculate the required values, and then scale the values during a second pass. 
* The problem is that this does not meet our requirement that the data only be used once.

* The way we do feature scaling in `river` involves calculating run statistics (runtime statistics). 
* The idea is that we use a data structure that estimates the mean and updates itself when given a value. The same is true for variance (and standard deviation). 
* For example, if $\mu_t$ represents the mean and $n_t$ represents the number of samples at time $t$, then the mean can be updated as follows:
$$n_{t+1} = n_t +1  \\
\mu_{t+1} = \mu_t + (x -\mu_t)/n_{t+1}$$

## Computation of the running mean (the variance)

In [5]:
n, mean, sum_of_squares, variance = 0, 0, 0, 0

def get_reg_dataset():
    california_housing = fetch_california_housing()
    features = california_housing.feature_names
    X = pd.DataFrame(california_housing.data, columns=features)
    y = pd.Series(california_housing.target)
    dataset = stream.iter_pandas(X, y)
    return dataset

for xi, yi in get_reg_dataset():
    n += 1
    old_mean = mean
    mean += (xi['Population'] - mean) / n
    sum_of_squares += (xi['Population'] - old_mean) * (xi['Population'] - mean)
    variance = sum_of_squares / n

print(f'Running mean: {mean:.3f}')
print(f'Running variance: {variance:.3f}')


Running mean: 1425.477
Running variance: 1282408.322


* For comparison, we calculate the classical (batch) values:

In [6]:
california_housing = fetch_california_housing()
features = california_housing.feature_names
X = pd.DataFrame(cal_housing.data, columns=cal_housing.feature_names)
print(f'True mean: {X["Population"].mean():.3f}')
print(f'True variance: {X["Population"].var():.3f}')


True mean: 1425.477
True variance: 1282470.457


* The results are similar: 
* The running statistics for the first observations are not very accurate. However, in general this does not matter too much. 
* The statistics can thus be updated when a new sample arrives. They can be used to scale the characteristics.
  * In `river` methods from the `StandardScaler` class are available for this purpose:

In [7]:
from river import preprocessing
scaler = preprocessing.StandardScaler()
for xi, yi in get_reg_dataset():
    scaler.learn_one(xi)

* In the following, we will implement a linear online classification task using logistic regression. 
* Since not all data is available at once, we have to perform the so-called stochastic gradient descent (SGD).
  * SGD is often used to train neural networks. 
  * The idea is that at each step we compute the loss between the target prediction and the truth. 
  * We then calculate the gradient, which is simply a set of derivatives with respect to each weight from the linear regression.
  * Once we have obtained the gradient, we can update the weights by moving them in the opposite direction of the gradient. 
  * The amount by which the weights are moved depends on a learning rate.  Different optimizers have different ways of managing the weight update, and some handle the learning rate implicitly.
  * Online linear regression can be done in `river` using the LinearRegression class from the `linear_model` module. 
* We simply use SGD with the SGD optimizer from the `optim` module. During the training we measure the squared error between the observed and the predicted values.

In [8]:
from river import linear_model
from river import optim
import statistics

scaler = preprocessing.StandardScaler()
optimizer = optim.SGD(lr=0.01)
log_reg = linear_model.LogisticRegression(optimizer)

y_true = []
y_pred = []

for xi, yi in get_dataset():
    
    # Before river 0.21.0:
    # Scale the features
    # xi_scaled = scaler.learn_one(xi).transform_one(xi)

    # After 0.21.0:
    scaler.learn_one(xi)
    xi_scaled = scaler.transform_one(xi)


    # Test the current model on the new "unobserved" sample
    yi_pred = log_reg.predict_proba_one(xi_scaled)
    # Train the model with the new sample
    log_reg.learn_one(xi_scaled, yi)

    # Store the truth and the prediction
    y_true.append(yi)
    y_pred.append(yi_pred[True])

print(f'Accuracy: {metrics.roc_auc_score(y_true, y_pred):.3f}')


y_95: 2.6472499999999997
Accuracy: 0.912


* Accuracy is better than that obtained from scikit-learn logistic regression cross-validation. 
* However, to make things truly comparable, it would be nice to compare using the same cross-validation procedure. 
* `river` has a compat module that contains utilities to make river compatible with other Python libraries. 
* Since we are doing regression, we will use `SKLRegressorWrapper`. 
* We will also use pipeline to encapsulate the logic of the StandardScaler and LogisticRegression in a single object.

* The following steps are implemented:
    * Define a river Pipeline, exactly like done earlier for sklearn 
    * Make the Pipeline compatible with sklearn
    * Compute the CV scores using the same CV scheme and the same scoring
    * Display the average score and it's standard deviation

In [9]:
from river import compat
from river import compose

model = compose.Pipeline(
    ('scale', preprocessing.StandardScaler()),
    ('log_reg', linear_model.LogisticRegression())
)
model = compat.convert_river_to_sklearn(model)
scores = model_selection.cross_val_score(model, X, y, scoring=scorer, cv=cv)
print(f'Accuracy: {scores.mean():.3f} (± {scores.std():.3f})')

Accuracy: 0.842 (± 0.006)


* Accuracy is lower this time, which we would expect. 
* In fact, online learning is not as accurate as batch learning. 
* However, it all depends on what you are interested in:
  * If you are only interested in predicting the next observation, the OML algorithm would be better. 
* For this reason, it is somewhat difficult to compare the two approaches: 
  * They are both suitable for different scenarios.

# Transformer in river

## One Hot Encoder

* We create a simple example data set.

In [10]:
from pprint import pprint
import random
import string

random.seed(42)
alphabet = list(string.ascii_lowercase)
X = [
    {
        'c1': random.choice(alphabet),
        'c2': random.choice(alphabet),
    }
    for _ in range(4)
]
pprint(X)

[{'c1': 'u', 'c2': 'd'},
 {'c1': 'a', 'c2': 'x'},
 {'c1': 'i', 'c2': 'h'},
 {'c1': 'h', 'c2': 'e'}]


* Apply `one-hot-encoding`:

In [11]:
from river import preprocessing

oh = preprocessing.OneHotEncoder()
for x in X:
    oh.learn_one(x)
    pprint(oh.transform_one(x))


{'c1_u': 1, 'c2_d': 1}
{'c1_a': 1, 'c1_u': 0, 'c2_d': 0, 'c2_x': 1}
{'c1_a': 0, 'c1_i': 1, 'c1_u': 0, 'c2_d': 0, 'c2_h': 1, 'c2_x': 0}
{'c1_a': 0,
 'c1_h': 1,
 'c1_i': 0,
 'c1_u': 0,
 'c2_d': 0,
 'c2_e': 1,
 'c2_h': 0,
 'c2_x': 0}


* A subset of the features can be one-hot coded using `compose.Select`.

In [12]:
from river import compose

pp = compose.Select('c1') | preprocessing.OneHotEncoder()

for x in X:
    pp.learn_one(x)
    pprint(pp.transform_one(x))


{'c1_u': 1}
{'c1_a': 1, 'c1_u': 0}
{'c1_a': 0, 'c1_i': 1, 'c1_u': 0}
{'c1_a': 0, 'c1_h': 1, 'c1_i': 0, 'c1_u': 0}
